## Check Hardware

In [10]:
!nvidia-smi

Fri Apr 22 15:29:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   42C    P8    22W / 260W |     21MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!pip list

Package                            Version
---------------------------------- -------------------
-andas                             1.1.3
absl-py                            0.14.1
alabaster                          0.7.12
albumentations                     0.4.6
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1

## Import Libraries

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import time
import seaborn as sns

from tqdm import tqdm
from pathlib import Path
from pyreadr import read_r
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import Dataset
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter

## Configs

### Paths

In [13]:
cwd = os.getcwd()
parent_dir = Path(cwd).parent.absolute()
root_dir = parent_dir.parent.absolute() 
CSV_PATH = os.path.join(root_dir, 'Dataset', 'csv', 'total_1.0.csv')
BEST_STATE_PATH = os.path.join(root_dir, 'Saved_models', 'Best_weight', 'best_state.pth')
BEST_MODEL_PATH = os.path.join(root_dir, 'Saved_models', 'Best_model', 'best_model.pth')

### Params

In [14]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
n_features = 10000
n_classes = 91
n_epochs = 30
train_batch_size = 8
val_batch_size = 16
FIRST_TIME = True

In [15]:
device

'cuda'

## Prepare Data

### Read From CSV Into Dataframes

In [16]:
df_total = pd.read_csv(CSV_PATH, index_col=0)
df_total.fillna(0)
df_total

,cg22054918,cg16476975,cg03940848,cg25570913,cg02809746,cg01517680,cg04110886,cg18222083,cg05649108,cg08141395,...,cg26505478,cg06075311,cg19145082,cg26691179,cg04842161,cg07621803,cg21100077,cg00703120,cg00540828,label
5684819014_R03C02,0.947269,0.008453,0.123884,0.963694,0.494731,0.028322,0.055772,0.014548,0.038964,0.850487,...,0.000000,0.209100,0.202847,0.225260,0.049959,0.949485,0.888074,0.679235,0.187460,"GBM, G34"
5684819014_R05C02,0.719027,0.067590,0.000000,0.885362,0.772570,0.704825,0.170992,0.000000,0.809885,0.044992,...,0.011448,0.090918,0.107293,0.170331,0.303312,0.932520,0.720423,0.812766,0.991197,"DMG, K27"
5684819014_R06C02,0.986904,0.000000,0.038046,1.000000,0.975408,0.000217,0.079723,0.715507,0.996868,0.000000,...,0.003699,0.292715,0.072444,0.454280,0.069150,0.926996,0.911045,0.844444,0.944158,"DMG, K27"
5684819013_R05C02,0.163068,0.231008,0.000000,0.947795,0.714640,0.032332,0.091954,0.000000,0.871492,0.013984,...,0.000000,0.085011,0.080590,0.303145,0.396163,0.645922,0.824696,0.680867,1.000000,"DMG, K27"
5684819014_R03C01,0.972470,0.119748,0.008277,0.984159,0.914704,0.433428,0.014778,0.000502,0.951393,0.037316,...,0.532789,0.398517,0.309837,0.129453,0.117355,0.093597,0.898011,0.917079,1.000000,"DMG, K27"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200091640051_R01C02,0.071838,0.788916,0.518146,0.073368,0.917277,0.908486,1.000000,0.738451,0.022102,0.896812,...,0.041958,0.834287,0.721935,0.846085,0.086545,0.425349,0.277112,0.253946,0.905899,"ENB, A"
200091640051_R04C02,0.879886,0.963159,0.829632,0.066611,0.929978,0.961191,0.862995,0.815590,0.902938,0.863093,...,0.027656,0.719758,0.199823,0.859725,0.248218,0.339218,0.764695,0.268875,0.842302,"ENB, A"
200091640051_R06C02,0.183778,0.528951,0.839423,0.931243,0.748958,0.950534,0.166527,0.924846,0.749935,0.458329,...,0.026221,0.737446,0.120581,0.700668,0.190458,0.935168,0.824159,0.871991,1.000000,"A IDH, HG"
200091640059_R02C01,0.036189,0.091265,0.030045,0.210802,0.017907,0.236162,0.620858,0.113647,0.197062,0.101033,...,0.044161,0.087165,0.159322,0.288011,0.754489,0.829186,0.532604,0.829158,0.960378,"LGG, DIG/DIA"


### Separate Features And Labels

In [17]:
features = np.array(df_total.iloc[:,:-1])
labels = np.array(df_total.iloc[:,-1])
print(features.shape, labels.shape, type(features), type(labels))

(2801, 10000) (2801,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [18]:
print(features[0], labels[0])

[0.94726853 0.00845275 0.12388393 ... 0.88807406 0.67923516 0.18746042] GBM, G34


### PCA

In [ ]:
features = PCA(n_components = 0.95).fit_transform(features)

In [ ]:
print(features.shape)

(2801, 10000)


In [ ]:
n_features = features.shape[1]
print(n_features)

10000


### Encode The Labels [1]

In [ ]:
# Label Encoding The Labels Ndarray
label_codes = {label:index for index, label in enumerate(np.unique(labels))}
print(label_codes)

{'A IDH': 0, 'A IDH, HG': 1, 'ANA PA': 2, 'ATRT, MYC': 3, 'ATRT, SHH': 4, 'ATRT, TYR': 5, 'CHGL': 6, 'CHORDM': 7, 'CN': 8, 'CNS NB, FOXR2': 9, 'CONTR, ADENOPIT': 10, 'CONTR, CEBM': 11, 'CONTR, HEMI': 12, 'CONTR, HYPTHAL': 13, 'CONTR, INFLAM': 14, 'CONTR, PINEAL': 15, 'CONTR, PONS': 16, 'CONTR, REACT': 17, 'CONTR, WM': 18, 'CPH, ADM': 19, 'CPH, PAP': 20, 'DLGNT': 21, 'DMG, K27': 22, 'EFT, CIC': 23, 'ENB, A': 24, 'ENB, B': 25, 'EPN, MPE': 26, 'EPN, PF A': 27, 'EPN, PF B': 28, 'EPN, RELA': 29, 'EPN, SPINE': 30, 'EPN, YAP': 31, 'ETMR': 32, 'EWS': 33, 'GBM, G34': 34, 'GBM, MES': 35, 'GBM, MID': 36, 'GBM, MYCN': 37, 'GBM, RTK I': 38, 'GBM, RTK II': 39, 'GBM, RTK III': 40, 'HGNET, BCOR': 41, 'HGNET, MN1': 42, 'HMB': 43, 'IHG': 44, 'LGG, DIG/DIA': 45, 'LGG, DNT': 46, 'LGG, GG': 47, 'LGG, MYB': 48, 'LGG, PA MID': 49, 'LGG, PA PF': 50, 'LGG, PA/GG ST': 51, 'LGG, RGNT': 52, 'LGG, SEGA': 53, 'LIPN': 54, 'LYMPHO': 55, 'MB, G3': 56, 'MB, G4': 57, 'MB, SHH CHL AD': 58, 'MB, SHH INF': 59, 'MB, WNT': 6

In [ ]:
def get_label_code (label, label_codes):
    if label in label_codes.keys():
        return label_codes[label]
    else:
        return -1 

print(get_label_code('CPH, PAP', label_codes), type(get_label_code('CPH, PAP', label_codes)))
print(get_label_code('fuqhfueh', label_codes))

20 <class 'int'>
-1


In [ ]:
encoded_labels = np.array([get_label_code(label, label_codes) for label in labels])
print(encoded_labels, encoded_labels.shape)

[34 22 22 ...  1 45  2] (2801,)


### Distribute Trainset and Testset

In [ ]:
(train_features, test_features, train_labels, test_labels) = train_test_split(features, encoded_labels, test_size = 0.05, random_state = 42)

In [ ]:
(train_features, val_features, train_labels, val_labels) = train_test_split(train_features, train_labels, test_size = 0.3, random_state = 42)

In [ ]:
print(train_features.shape, train_labels.shape, val_features.shape, val_labels.shape)

(1862, 10000) (1862,) (798, 10000) (798,)


### Random Over Sampling

In [ ]:
counter = Counter(train_labels)
print(counter)
over_sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
train_features, train_labels = over_sampler.fit_resample(train_features, train_labels)
counter = Counter(train_labels)
print(counter)

Counter({39: 104, 57: 93, 50: 77, 64: 64, 27: 62, 63: 56, 22: 52, 58: 51, 0: 51, 56: 48, 29: 43, 28: 39, 35: 38, 38: 37, 1: 34, 34: 31, 59: 30, 46: 30, 83: 28, 4: 28, 49: 28, 32: 28, 80: 26, 9: 25, 60: 24, 5: 22, 3: 21, 88: 20, 51: 20, 26: 20, 76: 19, 85: 19, 14: 17, 19: 17, 42: 17, 47: 16, 48: 16, 20: 16, 69: 15, 70: 15, 67: 15, 30: 15, 82: 15, 2: 14, 8: 14, 43: 14, 78: 13, 90: 13, 84: 13, 74: 13, 53: 13, 24: 12, 87: 12, 17: 12, 79: 11, 62: 11, 41: 11, 37: 11, 31: 10, 65: 10, 25: 10, 33: 10, 23: 9, 68: 9, 6: 9, 15: 9, 40: 8, 55: 8, 44: 8, 7: 8, 18: 8, 61: 8, 73: 8, 36: 8, 21: 7, 54: 7, 16: 7, 86: 7, 52: 6, 81: 6, 12: 6, 10: 6, 71: 6, 13: 5, 75: 5, 77: 5, 66: 5, 89: 5, 45: 5, 72: 3, 11: 2})
Counter({22: 104, 28: 104, 69: 104, 2: 104, 35: 104, 64: 104, 50: 104, 60: 104, 47: 104, 58: 104, 79: 104, 83: 104, 78: 104, 56: 104, 90: 104, 48: 104, 21: 104, 0: 104, 5: 104, 76: 104, 20: 104, 80: 104, 39: 104, 70: 104, 23: 104, 4: 104, 49: 104, 38: 104, 88: 104, 67: 104, 1: 104, 84: 104, 11: 104,

### Create Dataset Objects & Dataloader Objects

In [ ]:
class BTMD (Dataset):
    def __init__(self, features, labels, mode = 'train', split = None):
        self.mode = mode
        self.split = split
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        if self.mode == 'test':
            return torch.tensor(feature).float()
        else:
            return torch.tensor(feature).float(), torch.tensor(label).long()
    

In [ ]:
train_dataset = BTMD(train_features, train_labels, mode = 'Train')
val_dataset = BTMD(val_features, val_labels, mode = 'Val')
test_dataset = BTMD(test_features, test_labels, mode = 'Test')

In [ ]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

9464 798 141


In [ ]:
print(train_dataset[0])

(tensor([0.9285, 0.1503, 0.0140,  ..., 0.7881, 0.8023, 0.4802]), tensor(22))


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = train_batch_size, shuffle = True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = val_batch_size, shuffle = True, num_workers=3)

## Build Model

### Define Model Architecture

In [ ]:
class DNAMicroarrayNet (nn.Module):
    def __init__ (self, in_features, n_classes):
        super(DNAMicroarrayNet, self).__init__()
        self.in_features = in_features
        self.n_classes = n_classes
        self.densenet = nn.Sequential(
            nn.Linear(self.in_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, n_classes),
        )
        
    def forward (self, x):
        x = self.densenet(x)
        return x
        

In [ ]:
model = DNAMicroarrayNet(n_features, n_classes)
if FIRST_TIME == False:
    model.load_state_dict((torch.load(BEST_STATE_PATH)))

## Train And Validate Model

### Define Loss And Optimizer

In [ ]:
lr = 1e-4
weight_decay = 0

In [ ]:
criterion = CrossEntropyLoss(weight=None)
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

### Define Train And Val Pipeline

(remember to move tensors into GPU before feeding to neural network)

In [ ]:
def train_epoch(epoch, model, train_loader, criterion, optimizer, train_accs, train_losses):
    # print('Training: ')
    total_loss = 0
    correct = 0
    total = 0
    model.train()
    # For loop through all batches
    for features, labels in tqdm(train_loader):
        # Move tensors to GPU
        features = features.to(device)
        labels = labels.to(device)
        
        # Zero out gradient
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Evaluation
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
    # Averaging the loss for the whole epoch
    train_loss = total_loss / len(train_loader)
    train_acc = (correct / total) * 100.
    train_accs.append(train_acc)
    train_losses.append(train_loss)
    
    # print('train_loss: %.5f | train_acc: %.3f' % (train_loss, train_acc))
    return train_loss, train_acc
        
        
        

In [ ]:
def val_epoch(epoch, model, val_loader, criterion, val_accs, val_losses):
    # print('Validating: ')
    total_loss = 0
    correct = 0
    total = 0
    
    # For loop through all batches
    with torch.no_grad():
        # For loop through all batches
        for features, labels in tqdm(val_loader):
            # Move tensors to GPU
            features, labels = features.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)
            
            # Evaluation
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total  += labels.size(0)
        
        # Averaging the loss for the whole epoch
        val_loss = total_loss / len(val_loader)
        val_acc = (correct / total) * 100
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        # print('val_loss: %.5f | val_acc: %.3f' % (val_loss, val_acc))
    return val_loss, val_acc 
        

## Run

In [ ]:
history = {'train_accs': [], 'train_losses': [], 'val_accs': [], 'val_losses': []}
model.to(device)
diff_threshold = 1.0E-3
max_patience = 3
patience = 0

for epoch in range(1, n_epochs + 1):
    print(f'Epoch {epoch}/{n_epochs}: ')
    train_loss, train_acc = train_epoch(epoch, model, train_loader, criterion, optimizer, history['train_accs'], history['train_losses'])
    val_loss, val_acc = val_epoch(epoch, model, val_loader, criterion, history['val_accs'], history['val_losses'])
    
    print('train_loss: %.5f | train_acc: %.3f' % (train_loss, train_acc))
    print('val_loss: %.5f | val_acc: %.3f' % (val_loss, val_acc))
    
    if val_acc == max(history['val_accs']):
        print('Best validation accuracy => saving model weights...')
        torch.save(model.state_dict(), BEST_STATE_PATH)
        # print('Best validation accuracy => saving model...')
        # torch.save(model, BEST_MODEL_PATH)
        
    if len(history['val_accs']) > 1:
        if abs(history['val_accs'][-2] - val_acc) < diff_threshold or history['val_accs'][-2] > val_acc:
            patience = patience + 1
            print('Patience = {patience}')
            if patience == max_patience:
                print('Early stopping.')
                break
        else:
            patience = 0
    print('-------------------------------------------')

  0%|          | 0/1183 [00:00<?, ?it/s]

Epoch 1/30: 


  0%|          | 0/1183 [00:00<?, ?it/s]

train_loss: 3.84008 | train_acc: 12.658
val_loss: 2.71190 | val_acc: 33.835
Best validation accuracy => saving model weights...
-------------------------------------------
Epoch 2/30: 


  0%|          | 0/1183 [00:00<?, ?it/s]

...

train_loss: 0.00537 | train_acc: 99.926
val_loss: 0.13746 | val_acc: 95.614
Patience = {patience}
-------------------------------------------
Epoch 27/30: 


  0%|          | 0/1183 [00:00<?, ?it/s]

train_loss: 0.00300 | train_acc: 99.979
val_loss: 0.14495 | val_acc: 95.489
Patience = {patience}
-------------------------------------------
Epoch 28/30: 


100%|██████████| 50/50 [00:00<00:00, 348.16it/s]

train_loss: 0.00422 | train_acc: 99.894
val_loss: 0.14188 | val_acc: 95.363
Patience = {patience}
Early stopping.


## Predict on a test set

In [ ]:
# Predict one sample
rand = np.random.randint(0, len(test_dataset))
sample = test_dataset[rand]
print(sample)
sample_feature = sample[0].detach().cpu().numpy()
sample_feature = torch.tensor(sample_feature)
sample_feature = sample_feature.to(device)

# Check result
result = model(sample_feature).detach().cpu().numpy()
print(f'Result not through softmax: {np.argmax(result)}')

# Add softmax for probability calculation
softmax_result = nn.Softmax(dim=0)(torch.tensor(result)).detach().cpu().numpy()
print(f'Result passed through softmax: {np.argmax(softmax_result)} with probability {softmax_result[np.argmax(softmax_result)]}')

(tensor([0.0417, 0.0167, 0.0165,  ..., 0.4165, 0.8687, 0.9729]), tensor(33))
Result not through softmax: 33
Result passed through softmax: 33 with probability 0.9895638823509216


In [ ]:
# Predict on the whole test set
test_correct = 0
test_total = len(test_dataset)
for index in range(test_total):
    sample = test_dataset[index]
    sample_feature = sample[0]
    sample_label = sample[1]
    sample_feature = sample_feature.to(device)
    result = model(sample_feature).detach().cpu().numpy()
    softmax_result = nn.Softmax(dim=0)(torch.tensor(result)).detach().cpu().numpy()
    predict = np.argmax(softmax_result)
    if predict == sample[1]:
        test_correct += 1

print(test_correct/test_total)

0.9716312056737588


# References

\[1\]: [Is One-Hot Encoding required for using PyTorch's Cross Entropy Loss Function?](https://stackoverflow.com/questions/62456558/is-one-hot-encoding-required-for-using-pytorchs-cross-entropy-loss-function)

\[2\]: [PCA — how to choose the number of components?](https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/)

\[3\]: [Change weight decay during training](https://discuss.pytorch.org/t/change-weight-decay-during-training/70377/3)